In [ ]:
# Importar la librería pandas
import pandas as pd

# Especifica la ruta completa del archivo
ruta_archivo = r'C:\Users\jdvil\Documents\Python\NAACL\Task_9\starting_k\incidents_labelled.csv'

# Leer el archivo CSV
carga = pd.read_csv(ruta_archivo, encoding='ISO-8859-15')

# Mostrar las primeras filas del DataFrame
carga.head()

In [ ]:
carga.count()

In [5]:
df = carga

In [ ]:
# Revisar datos faltantes en cada columna
missing_data = df.isnull().sum()
print("Datos faltantes por columna:")
print(missing_data)

In [ ]:
import re
from collections import Counter

# Definir función para extraer caracteres especiales
def extract_special_characters(column):
    special_characters = re.findall(r'[^a-zA-Z0-9\s]', ''.join(column))
    return Counter(special_characters)

# Obtener caracteres especiales y su concurrencia en las columnas 'title' y 'text'
special_characters_title = extract_special_characters(df['title'].astype(str))
special_characters_text = extract_special_characters(df['text'].astype(str))

print("Caracteres especiales en la columna 'title':")
print(special_characters_title)

print("\nCaracteres especiales en la columna 'text':")
print(special_characters_text)

In [ ]:
import re

# Función para limpiar texto ignorando errores de codificación
def clean_text(text):
    # Intentar decodificar correctamente con UTF-8, ignorando errores
    try:
        # Encode the text to bytes, then decode using utf-8
        text = text.encode('latin1', errors='ignore').decode('utf-8', errors='ignore')
    except (UnicodeDecodeError, AttributeError):
        pass
    # Remover cualquier otro carácter especial no ASCII
    return re.sub(r'[^\x00-\x7F]+', '', text)

# Aplicar la limpieza a las columnas 'title' y 'text'
df['title_clean'] = df['title'].apply(clean_text)
df['text_clean'] = df['text'].apply(clean_text)

# Mostrar las primeras filas de las columnas limpiadas
df[['title_clean', 'text_clean']].head()

In [ ]:
import re
from collections import Counter

# Función para extraer caracteres especiales
def extract_special_characters(column):
    special_characters = re.findall(r'[^a-zA-Z0-9\s]', ''.join(column))
    return Counter(special_characters)

# Revisar nuevamente los caracteres especiales en las columnas limpiadas 'title_clean' y 'text_clean'
special_characters_title_clean = extract_special_characters(df['title_clean'].astype(str))
special_characters_text_clean = extract_special_characters(df['text_clean'].astype(str))

# Mostrar los resultados
print("Caracteres especiales en la columna 'title_clean':")
print(special_characters_title_clean)

print("\nCaracteres especiales en la columna 'text_clean':")
print(special_characters_text_clean)

In [ ]:
# Eliminar los saltos de línea (\n) en las columnas 'title_clean' y 'text_clean'
df['title_clean'] = df['title_clean'].str.replace('\n', ' ')
df['text_clean'] = df['text_clean'].str.replace('\n', ' ')

# Mostrar las primeras filas para verificar
df[['title_clean', 'text_clean']].head()

In [ ]:
df.head()

In [ ]:
# Revisar la distribución de las columnas objetivo: hazard-category y product-category
hazard_distribution = df['hazard-category'].value_counts()
product_distribution = df['product-category'].value_counts()

# Mostrar la distribución
print("Distribución de 'hazard-category':")
print(hazard_distribution)

print("\nDistribución de 'product-category':")
print(product_distribution)

In [ ]:
# Contar cuántos hazard-category únicos tiene cada product-category
product_hazard_mapping = df.groupby('product-category')['hazard-category'].nunique()

# Filtrar los productos que están presentes en más de un hazard-category
products_in_multiple_hazards = product_hazard_mapping[product_hazard_mapping > 1]

# Mostrar los productos con más de un hazard-category
print("Productos presentes en más de un hazard-category:")
print(products_in_multiple_hazards)

# Mostrar el detalle de los productos con sus hazard-category
for product in products_in_multiple_hazards.index:
    print(f"\nProduct: {product}")
    # Filtrar por cada producto para ver las categorías de riesgo asociadas
    hazards_for_product = df[df['product-category'] == product]['hazard-category'].unique()
    print(f"Hazard categories: {hazards_for_product}")

In [ ]:
# Contar cuántas veces cada combinación de 'product-category' y 'hazard-category' ocurre
product_hazard_counts = df.groupby(['product-category', 'hazard-category']).size().reset_index(name='count')

# Mostrar el resultado
print(product_hazard_counts)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear una tabla pivote para visualizar mejor la relación entre productos y riesgos
pivot_table = df.groupby(['product-category', 'hazard-category']).size().unstack(fill_value=0)

# Crear un heatmap con seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt="d", cmap="YlGnBu")
plt.title('Relación entre Product Category y Hazard Category')
plt.ylabel('Product Category')
plt.xlabel('Hazard Category')
plt.show()

In [ ]:
# Obtener los países únicos en la columna 'country'
unique_countries = df['country'].unique()

# Mostrar la lista de países únicos
print("Países únicos en el DataFrame:")
print(unique_countries)

In [ ]:
# Agrupar por year, country, y product-category, luego contar los hazard-category únicos
product_hazard_per_location = df.groupby(['year', 'country', 'product-category'])['hazard-category'].nunique().reset_index()

# Filtrar aquellos casos donde un product-category está presente en más de un hazard-category
products_in_multiple_hazards_location = product_hazard_per_location[product_hazard_per_location['hazard-category'] > 1]

# Mostrar los resultados
print("Productos que están presentes en múltiples hazard-category en la misma combinación de año y país:")
print(products_in_multiple_hazards_location)

In [ ]:
import matplotlib.pyplot as plt

# Crear un gráfico de barras por país
country_summary = products_in_multiple_hazards_location.groupby('country').size().reset_index(name='num_products')

plt.figure(figsize=(10,6))
plt.bar(country_summary['country'], country_summary['num_products'])
plt.title('Número de productos en múltiples hazard-category por país')
plt.xlabel('País')
plt.ylabel('Número de productos')
plt.show()

# Modelo Base

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Suponiendo que tu DataFrame se llama df
# Paso 1: Combinar las columnas 'title_clean' y 'text_clean' en una sola
df['combined_text'] = df['text_clean']

# Paso 2: Codificar las etiquetas
# Codificar 'hazard-category'
le_hazard = LabelEncoder()
df['hazard_encoded'] = le_hazard.fit_transform(df['hazard-category'])

# Codificar 'product-category' dentro de cada 'hazard-category'
# Crear un diccionario para almacenar los codificadores de 'product-category' por 'hazard-category'
product_encoders = {}
df['product_encoded'] = -1  # Inicializar con -1

# Obtener las categorías únicas de 'hazard-category'
hazard_categories = df['hazard-category'].unique()

for hazard in hazard_categories:
    # Filtrar el DataFrame por 'hazard-category'
    mask = df['hazard-category'] == hazard
    # Crear un LabelEncoder para el 'hazard-category' actual
    le_product = LabelEncoder()
    df.loc[mask, 'product_encoded'] = le_product.fit_transform(df.loc[mask, 'product-category'])
    # Almacenar el codificador
    product_encoders[hazard] = le_product

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X = df['combined_text']
y_hazard = df['hazard_encoded']
y_product = df['product_encoded']

X_train, X_test, y_hazard_train, y_hazard_test, y_product_train, y_product_test = train_test_split(
    X, y_hazard, y_product, test_size=0.2, random_state=42, stratify=y_hazard
)

# Paso 4: Crear un Pipeline para el modelo de SVM
from sklearn.base import BaseEstimator, TransformerMixin

# Crear un Pipeline para 'hazard-category'
pipeline_hazard = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LinearSVC())
])

# Entrenar el modelo para 'hazard-category'
pipeline_hazard.fit(X_train, y_hazard_train)

# Predecir en el conjunto de prueba
y_hazard_pred = pipeline_hazard.predict(X_test)

# Paso 5: Evaluar el modelo para 'hazard-category'
print("Evaluación del modelo para 'hazard-category':")
print(classification_report(y_hazard_test, y_hazard_pred, target_names=le_hazard.classes_))
print("Matriz de confusión:")
print(confusion_matrix(y_hazard_test, y_hazard_pred))
print("Accuracy:", accuracy_score(y_hazard_test, y_hazard_pred))
print("F1 Score:", f1_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Precision:", precision_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Recall:", recall_score(y_hazard_test, y_hazard_pred, average='weighted'))

# Paso 6: Para 'product-category', necesitamos tener en cuenta la jerarquía
# Crear un diccionario para almacenar los modelos por 'hazard-category'
product_models = {}

# Entrenar un modelo para cada 'hazard-category'
for hazard in hazard_categories:
    # Filtrar los datos correspondientes
    hazard_idx = le_hazard.transform([hazard])[0]
    mask_train = y_hazard_train == hazard_idx
    # Verificar si hay suficientes muestras para entrenar
    if np.sum(mask_train) > 1:
        X_train_hazard = X_train[mask_train]
        y_product_train_hazard = y_product_train[mask_train]

        # Crear y entrenar el modelo
        pipeline_product = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('classifier', LinearSVC())
        ])
        pipeline_product.fit(X_train_hazard, y_product_train_hazard)
        # Almacenar el modelo
        product_models[hazard] = pipeline_product

# Paso 7: Predecir 'product-category' teniendo en cuenta 'hazard-category'
y_product_pred = []

for i in range(len(X_test)):
    hazard_idx = y_hazard_pred[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]

    # Si tenemos un modelo para este 'hazard-category'
    if hazard_label in product_models:
        model = product_models[hazard_label]
        # Transformar el texto en una lista para el modelo
        X_sample = [X_test.iloc[i]]
        # Predecir 'product-category'
        product_pred_encoded = model.predict(X_sample)[0]
        # Decodificar la predicción
        le_product = product_encoders[hazard_label]
        product_pred_label = le_product.inverse_transform([product_pred_encoded])[0]
        # Añadir a la lista de predicciones
        y_product_pred.append((hazard_label, product_pred_label))
    else:
        # Si no hay modelo, asignar una etiqueta por defecto
        y_product_pred.append((hazard_label, None))

# Paso 8: Evaluar el modelo para 'product-category'
# Necesitamos comparar las predicciones con las etiquetas verdaderas
y_product_true_labels = []
y_product_pred_labels = []

for i in range(len(y_product_test)):
    hazard_idx = y_hazard_test.iloc[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]
    product_idx = y_product_test.iloc[i]
    le_product = product_encoders[hazard_label]
    product_label = le_product.inverse_transform([product_idx])[0]

    y_product_true_labels.append((hazard_label, product_label))

    # Obtener la predicción correspondiente
    pred_hazard_label, pred_product_label = y_product_pred[i]
    y_product_pred_labels.append((pred_hazard_label, pred_product_label))

# Evaluar solo las instancias donde tenemos predicciones válidas
true_labels = []
pred_labels = []

for true, pred in zip(y_product_true_labels, y_product_pred_labels):
    # Si la predicción no es None
    if pred[1] is not None:
        true_labels.append(true[1])
        pred_labels.append(pred[1])

# Reporte de clasificación para 'product-category'
print("\nEvaluación del modelo para 'product-category':")
print(classification_report(true_labels, pred_labels))
print("Matriz de confusión:")
print(confusion_matrix(true_labels, pred_labels))
print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("F1 Score:", f1_score(true_labels, pred_labels, average='weighted'))
print("Precision:", precision_score(true_labels, pred_labels, average='weighted'))
print("Recall:", recall_score(true_labels, pred_labels, average='weighted'))


# Mejorado Jerarquia y nuevos campos

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Paso 1: Combinar las columnas 'title_clean' y 'text_clean' en una sola
df['combined_text'] = df['text_clean']

# Paso 2: Codificar las etiquetas
# Codificar 'hazard-category'
le_hazard = LabelEncoder()
df['hazard_encoded'] = le_hazard.fit_transform(df['hazard-category'])

# Codificar 'product-category' dentro de cada 'hazard-category'
# Crear un diccionario para almacenar los codificadores de 'product-category' por 'hazard-category'
product_encoders = {}
df['product_encoded'] = -1  # Inicializar con -1

# Obtener las categorías únicas de 'hazard-category'
hazard_categories = df['hazard-category'].unique()

for hazard in hazard_categories:
    # Filtrar el DataFrame por 'hazard-category'
    mask = df['hazard-category'] == hazard
    # Crear un LabelEncoder para el 'hazard-category' actual
    le_product = LabelEncoder()
    df.loc[mask, 'product_encoded'] = le_product.fit_transform(df.loc[mask, 'product-category'])
    # Almacenar el codificador
    product_encoders[hazard] = le_product

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X = df['combined_text']
y_hazard = df['hazard_encoded']
y_product = df['product_encoded']

X_train, X_test, y_hazard_train, y_hazard_test, y_product_train, y_product_test = train_test_split(
    X, y_hazard, y_product, test_size=0.2, random_state=42, stratify=y_hazard
)

# Paso 4: Crear un Pipeline para el modelo de SVM
from sklearn.base import BaseEstimator, TransformerMixin

# Crear un Pipeline para 'hazard-category'
pipeline_hazard = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LinearSVC())
])

# Entrenar el modelo para 'hazard-category'
pipeline_hazard.fit(X_train, y_hazard_train)

# Predecir en el conjunto de prueba
y_hazard_pred = pipeline_hazard.predict(X_test)

# Función para buscar hazard-categories válidos por año y país, retrocediendo si no hay datos
def find_valid_hazard_categories(year, country, product_hazard_per_location):
    while year >= product_hazard_per_location['year'].min():  # Limitar búsqueda al mínimo año disponible
        # Filtrar por el año y país actual
        valid_hazards = product_hazard_per_location[
            (product_hazard_per_location['year'] == year) &
            (product_hazard_per_location['country'] == country)
        ]

        if not valid_hazards.empty:
            return valid_hazards['hazard-category'].unique()  # Devolver los hazard-category válidos si hay datos

        year -= 1  # Retroceder al año anterior si no se encontraron datos

    return None  # Si no se encuentra nada, devolver None

# Función para encontrar el producto con más peso para un hazard-category dado en un año
def find_most_frequent_product_for_hazard(hazard_label, year, country, df):
    # Filtrar el DataFrame por el hazard-category y el año específico
    mask = (df['hazard-category'] == hazard_label) & (df['year'] == year) & (df['country'] == country)
    filtered_df = df[mask]

    if not filtered_df.empty:
        # Encontrar el producto más frecuente en ese hazard-category
        most_frequent_product = filtered_df['product-category'].mode()[0]  # Devuelve el producto más frecuente
        return most_frequent_product
    else:
        # Si no hay productos en ese año, retrocedemos para buscar el producto más frecuente históricamente
        mask_all_years = df['hazard-category'] == hazard_label
        filtered_df_all_years = df[mask_all_years]
        if not filtered_df_all_years.empty:
            most_frequent_product = filtered_df_all_years['product-category'].mode()[0]
            return most_frequent_product
        else:
            return None  # Si no hay datos históricos, devolver None

# Paso 5: Evaluar el modelo para 'hazard-category'
print("Evaluación del modelo para 'hazard-category':")
print(classification_report(y_hazard_test, y_hazard_pred, target_names=le_hazard.classes_))
print("Matriz de confusión:")
print(confusion_matrix(y_hazard_test, y_hazard_pred))
print("Accuracy:", accuracy_score(y_hazard_test, y_hazard_pred))
print("F1 Score:", f1_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Precision:", precision_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Recall:", recall_score(y_hazard_test, y_hazard_pred, average='weighted'))

# Paso 6: Para 'product-category', necesitamos tener en cuenta la jerarquía
# Crear un diccionario para almacenar los modelos por 'hazard-category'
product_models = {}

# Entrenar un modelo para cada 'hazard-category'
for hazard in hazard_categories:
    # Filtrar los datos correspondientes
    hazard_idx = le_hazard.transform([hazard])[0]
    mask_train = y_hazard_train == hazard_idx
    # Verificar si hay suficientes muestras para entrenar
    if np.sum(mask_train) > 1:
        X_train_hazard = X_train[mask_train]
        y_product_train_hazard = y_product_train[mask_train]

        # Crear y entrenar el modelo
        pipeline_product = Pipeline([
            ('vectorizer', CountVectorizer()),
            ('classifier', LinearSVC())
        ])
        pipeline_product.fit(X_train_hazard, y_product_train_hazard)
        # Almacenar el modelo
        product_models[hazard] = pipeline_product

# Paso 7: Predecir 'product-category' teniendo en cuenta 'hazard-category' y validación por año/país
y_product_pred = []

# product_hazard_per_location tiene todas las combinaciones de year, country y hazard
product_hazard_per_location = df.groupby(['year', 'country', 'product-category'])['hazard-category'].nunique().reset_index()

for i in range(len(X_test)):
    hazard_idx = y_hazard_pred[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]

    # Si tenemos un modelo para este 'hazard-category'
    if hazard_label in product_models:
        model = product_models[hazard_label]
        # Transformar el texto en una lista para el modelo
        X_sample = [X_test.iloc[i]]
        # Predecir 'product-category'
        product_pred_encoded = model.predict(X_sample)[0]
        # Decodificar la predicción
        le_product = product_encoders[hazard_label]
        product_pred_label = le_product.inverse_transform([product_pred_encoded])[0]
        # Añadir a la lista de predicciones
        y_product_pred.append((hazard_label, product_pred_label))
    else:
        # Si no hay modelo, validar año y país, y asignar un hazard válido
        year_pred = df['year'].iloc[i]  # Año actual
        country_pred = df['country'].iloc[i]  # País actual

        valid_hazard_categories = find_valid_hazard_categories(year_pred, country_pred, product_hazard_per_location)

        if valid_hazard_categories is not None:
            # Devolver el producto con más peso para este hazard-category en ese año/país
            most_frequent_product = find_most_frequent_product_for_hazard(hazard_label, year_pred, country_pred, df)
            y_product_pred.append((hazard_label, most_frequent_product))
        else:
            # Si no se encuentran categorías válidas, asignar el producto más frecuente para ese hazard
            most_frequent_product = find_most_frequent_product_for_hazard(hazard_label, year_pred, country_pred, df)
            y_product_pred.append((hazard_label, most_frequent_product))

# Paso 8: Evaluar el modelo para 'product-category'
y_product_true_labels = []
y_product_pred_labels = []

for i in range(len(y_product_test)):
    hazard_idx = y_hazard_test.iloc[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]
    product_idx = y_product_test.iloc[i]
    le_product = product_encoders[hazard_label]
    product_label = le_product.inverse_transform([product_idx])[0]

    y_product_true_labels.append((hazard_label, product_label))

    # Obtener la predicción correspondiente
    pred_hazard_label, pred_product_label = y_product_pred[i]
    y_product_pred_labels.append((pred_hazard_label, pred_product_label))

# Evaluar solo las instancias donde tenemos predicciones válidas
true_labels = []
pred_labels = []

for true, pred in zip(y_product_true_labels, y_product_pred_labels):
    # Si la predicción no es None
    if pred[1] is not None:
        true_labels.append(true[1])
        pred_labels.append(pred[1])

# Reporte de clasificación para 'product-category'
print("\nEvaluación del modelo para 'product-category':")
print(classification_report(true_labels, pred_labels))
print("Matriz de confusión:")
print(confusion_matrix(true_labels, pred_labels))
print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("F1 Score:", f1_score(true_labels, pred_labels, average='weighted'))
print("Precision:", precision_score(true_labels, pred_labels, average='weighted'))
print("Recall:", recall_score(true_labels, pred_labels, average='weighted'))

In [ ]:
# Crear listas para almacenar las predicciones y las etiquetas verdaderas
hazard_category_true = []
hazard_category_pred = []
product_category_true = []
product_category_pred = []
years = []
countries = []
texts_clean = []

# Iterar sobre el conjunto de datos de prueba y almacenar los resultados
for i in range(len(y_product_test)):
    # Obtener las etiquetas verdaderas para hazard-category y product-category
    hazard_idx_true = y_hazard_test.iloc[i]
    hazard_label_true = le_hazard.inverse_transform([hazard_idx_true])[0]

    product_idx_true = y_product_test.iloc[i]
    le_product = product_encoders[hazard_label_true]
    product_label_true = le_product.inverse_transform([product_idx_true])[0]

    # Obtener las predicciones correspondientes
    hazard_label_pred, product_label_pred = y_product_pred[i]

    # Obtener los valores adicionales
    year = df['year'].iloc[i]
    country = df['country'].iloc[i]
    text_clean = df['text_clean'].iloc[i]

    # Almacenar los valores en las listas
    hazard_category_true.append(hazard_label_true)
    hazard_category_pred.append(hazard_label_pred)
    product_category_true.append(product_label_true)
    product_category_pred.append(product_label_pred)
    years.append(year)
    countries.append(country)
    texts_clean.append(text_clean)

# Crear el DataFrame con los resultados y los campos adicionales
df_resultados_completo = pd.DataFrame({
    'year': years,
    'country': countries,
    'text_clean': texts_clean,
    'hazard_category_true': hazard_category_true,
    'hazard_category_pred': hazard_category_pred,
    'product_category_true': product_category_true,
    'product_category_pred': product_category_pred
})

# Mostrar el DataFrame de resultados completo
print(df_resultados_completo)

In [ ]:
# # Guardar el DataFrame en un archivo Excel
# df_resultados_completo.to_excel('resultados_prediccion.xlsx', index=False)

# # Descargar el archivo a tu máquina local
# from google.colab import files
# files.download('resultados_prediccion.xlsx')

# Modelos de Jerarquia mas COmplejo

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder


# Paso 1: Combinar las columnas 'title_clean' y 'text_clean' en una sola
df['combined_text'] = df['text_clean']

# Paso 2: Codificar las etiquetas
# Codificar 'hazard-category'
le_hazard = LabelEncoder()
df['hazard_encoded'] = le_hazard.fit_transform(df['hazard-category'])

# Codificar 'product-category' dentro de cada 'hazard-category'
# Crear un diccionario para almacenar los codificadores de 'product-category' por 'hazard-category'
product_encoders = {}
df['product_encoded'] = -1  # Inicializar con -1

# Obtener las categorías únicas de 'hazard-category'
hazard_categories = df['hazard-category'].unique()

for hazard in hazard_categories:
    # Filtrar el DataFrame por 'hazard-category'
    mask = df['hazard-category'] == hazard
    # Crear un LabelEncoder para el 'hazard-category' actual
    le_product = LabelEncoder()
    df.loc[mask, 'product_encoded'] = le_product.fit_transform(df.loc[mask, 'product-category'])
    # Almacenar el codificador
    product_encoders[hazard] = le_product

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X = df['combined_text']
y_hazard = df['hazard_encoded']
y_product = df['product_encoded']

X_train, X_test, y_hazard_train, y_hazard_test, y_product_train, y_product_test = train_test_split(
    X, y_hazard, y_product, test_size=0.2, random_state=42, stratify=y_hazard
)

# Paso 4: Crear un Pipeline para el modelo de SVM
from sklearn.base import BaseEstimator, TransformerMixin

# Crear un Pipeline para 'hazard-category' con TfidfVectorizer
pipeline_hazard = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LinearSVC())
])

# Entrenar el modelo para 'hazard-category'
pipeline_hazard.fit(X_train, y_hazard_train)

# Predecir en el conjunto de prueba
y_hazard_pred = pipeline_hazard.predict(X_test)

# Función para buscar hazard-categories válidos por año y país, retrocediendo si no hay datos
def find_valid_hazard_categories(year, country, product_hazard_per_location):
    while year >= product_hazard_per_location['year'].min():  # Limitar búsqueda al mínimo año disponible
        # Filtrar por el año y país actual
        valid_hazards = product_hazard_per_location[
            (product_hazard_per_location['year'] == year) &
            (product_hazard_per_location['country'] == country)
        ]

        if not valid_hazards.empty:
            return valid_hazards['hazard-category'].unique()  # Devolver los hazard-category válidos si hay datos

        year -= 1  # Retroceder al año anterior si no se encontraron datos

    return None  # Si no se encuentra nada, devolver None

# Función para encontrar el producto con más peso para un hazard-category dado en un año
def find_most_frequent_product_for_hazard(hazard_label, year, country, df):
    # Filtrar el DataFrame por el hazard-category y el año específico
    mask = (df['hazard-category'] == hazard_label) & (df['year'] == year) & (df['country'] == country)
    filtered_df = df[mask]

    if not filtered_df.empty:
        # Encontrar el producto más frecuente en ese hazard-category
        most_frequent_product = filtered_df['product-category'].mode()[0]  # Devuelve el producto más frecuente
        return most_frequent_product
    else:
        # Si no hay productos en ese año, retrocedemos para buscar el producto más frecuente históricamente
        mask_all_years = df['hazard-category'] == hazard_label
        filtered_df_all_years = df[mask_all_years]
        if not filtered_df_all_years.empty:
            most_frequent_product = filtered_df_all_years['product-category'].mode()[0]
            return most_frequent_product
        else:
            return None  # Si no hay datos históricos, devolver None

# Paso 5: Evaluar el modelo para 'hazard-category'
print("Evaluación del modelo para 'hazard-category':")
print(classification_report(y_hazard_test, y_hazard_pred, target_names=le_hazard.classes_))
print("Matriz de confusión:")
print(confusion_matrix(y_hazard_test, y_hazard_pred))
print("Accuracy:", accuracy_score(y_hazard_test, y_hazard_pred))
print("F1 Score:", f1_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Precision:", precision_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Recall:", recall_score(y_hazard_test, y_hazard_pred, average='weighted'))

# Paso 6: Para 'product-category', necesitamos tener en cuenta la jerarquía
# Crear un diccionario para almacenar los modelos por 'hazard-category'
product_models = {}

# Entrenar un modelo para cada 'hazard-category'
for hazard in hazard_categories:
    # Filtrar los datos correspondientes
    hazard_idx = le_hazard.transform([hazard])[0]
    mask_train = y_hazard_train == hazard_idx
    # Verificar si hay suficientes muestras para entrenar
    if np.sum(mask_train) > 1:
        X_train_hazard = X_train[mask_train]
        y_product_train_hazard = y_product_train[mask_train]

        # Crear y entrenar el modelo con TfidfVectorizer
        pipeline_product = Pipeline([
            ('vectorizer', TfidfVectorizer()),
            ('classifier', LinearSVC())
        ])
        pipeline_product.fit(X_train_hazard, y_product_train_hazard)
        # Almacenar el modelo
        product_models[hazard] = pipeline_product

# Paso 7: Predecir 'product-category' teniendo en cuenta 'hazard-category' y validación por año/país
y_product_pred = []

# product_hazard_per_location tiene todas las combinaciones de year, country y hazard
product_hazard_per_location = df.groupby(['year', 'country', 'product-category'])['hazard-category'].nunique().reset_index()

for i in range(len(X_test)):
    hazard_idx = y_hazard_pred[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]

    # Si tenemos un modelo para este 'hazard-category'
    if hazard_label in product_models:
        model = product_models[hazard_label]
        # Transformar el texto en una lista para el modelo
        X_sample = [X_test.iloc[i]]
        # Predecir 'product-category'
        product_pred_encoded = model.predict(X_sample)[0]
        # Decodificar la predicción
        le_product = product_encoders[hazard_label]
        product_pred_label = le_product.inverse_transform([product_pred_encoded])[0]
        # Añadir a la lista de predicciones
        y_product_pred.append((hazard_label, product_pred_label))
    else:
        # Si no hay modelo, validar año y país, y asignar un hazard válido
        year_pred = df['year'].iloc[i]  # Año actual
        country_pred = df['country'].iloc[i]  # País actual

        valid_hazard_categories = find_valid_hazard_categories(year_pred, country_pred, product_hazard_per_location)

        if valid_hazard_categories is not None:
            # Devolver el producto con más peso para este hazard-category en ese año/país
            most_frequent_product = find_most_frequent_product_for_hazard(hazard_label, year_pred, country_pred, df)
            y_product_pred.append((hazard_label, most_frequent_product))
        else:
            # Si no se encuentran categorías válidas, asignar el producto más frecuente para ese hazard
            most_frequent_product = find_most_frequent_product_for_hazard(hazard_label, year_pred, country_pred, df)
            y_product_pred.append((hazard_label, most_frequent_product))

# Paso 8: Evaluar el modelo para 'product-category'
y_product_true_labels = []
y_product_pred_labels = []

for i in range(len(y_product_test)):
    hazard_idx = y_hazard_test.iloc[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]
    product_idx = y_product_test.iloc[i]
    le_product = product_encoders[hazard_label]
    product_label = le_product.inverse_transform([product_idx])[0]

    y_product_true_labels.append((hazard_label, product_label))

    # Obtener la predicción correspondiente
    pred_hazard_label, pred_product_label = y_product_pred[i]
    y_product_pred_labels.append((pred_hazard_label, pred_product_label))

# Evaluar solo las instancias donde tenemos predicciones válidas
true_labels = []
pred_labels = []

for true, pred in zip(y_product_true_labels, y_product_pred_labels):
    # Si la predicción no es None
    if pred[1] is not None:
        true_labels.append(true[1])
        pred_labels.append(pred[1])

# Reporte de clasificación para 'product-category'
print("\nEvaluación del modelo para 'product-category':")
print(classification_report(true_labels, pred_labels))
print("Matriz de confusión:")
print(confusion_matrix(true_labels, pred_labels))
print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("F1 Score:", f1_score(true_labels, pred_labels, average='weighted'))
print("Precision:", precision_score(true_labels, pred_labels, average='weighted'))
print("Recall:", recall_score(true_labels, pred_labels, average='weighted'))

# Hiperparametros

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Suponiendo que tu DataFrame se llama df
# Paso 1: Combinar las columnas 'title_clean' y 'text_clean' en una sola
df['combined_text'] = df['text_clean']

# Paso 2: Codificar las etiquetas
# Codificar 'hazard-category'
le_hazard = LabelEncoder()
df['hazard_encoded'] = le_hazard.fit_transform(df['hazard-category'])

# Codificar 'product-category' dentro de cada 'hazard-category'
# Crear un diccionario para almacenar los codificadores de 'product-category' por 'hazard-category'
product_encoders = {}
df['product_encoded'] = -1  # Inicializar con -1

# Obtener las categorías únicas de 'hazard-category'
hazard_categories = df['hazard-category'].unique()

for hazard in hazard_categories:
    # Filtrar el DataFrame por 'hazard-category'
    mask = df['hazard-category'] == hazard
    # Crear un LabelEncoder para el 'hazard-category' actual
    le_product = LabelEncoder()
    df.loc[mask, 'product_encoded'] = le_product.fit_transform(df.loc[mask, 'product-category'])
    # Almacenar el codificador
    product_encoders[hazard] = le_product

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X = df['combined_text']
y_hazard = df['hazard_encoded']
y_product = df['product_encoded']

X_train, X_test, y_hazard_train, y_hazard_test, y_product_train, y_product_test = train_test_split(
    X, y_hazard, y_product, test_size=0.2, random_state=42, stratify=y_hazard
)

# Paso 4: Crear un Pipeline para el modelo de SVM con TfidfVectorizer y búsqueda de hiperparámetros
pipeline_hazard = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LinearSVC())
])

# Definir el espacio de búsqueda de hiperparámetros
param_grid_hazard = {
    'vectorizer__max_df': [0.75, 0.85, 1.0],
    'vectorizer__min_df': [1, 2, 5],
    'vectorizer__ngram_range': [(1,1), (1,2)],
    'classifier__C': [0.01, 0.1, 1, 10],
}

# Crear GridSearchCV para 'hazard-category'
grid_search_hazard = GridSearchCV(pipeline_hazard, param_grid_hazard, cv=5, scoring='f1_weighted', verbose=1, n_jobs=-1)

# Ajustar el modelo de 'hazard-category' con búsqueda de hiperparámetros
grid_search_hazard.fit(X_train, y_hazard_train)

# Obtener el mejor modelo para 'hazard-category'
best_model_hazard = grid_search_hazard.best_estimator_

# Predecir en el conjunto de prueba para 'hazard-category'
y_hazard_pred = best_model_hazard.predict(X_test)

# Evaluar el modelo de 'hazard-category'
print("Evaluación del modelo para 'hazard-category' tras la optimización:")
print(classification_report(y_hazard_test, y_hazard_pred, target_names=le_hazard.classes_))

# Paso 5: Crear un pipeline jerárquico para 'product-category'
product_models = {}

# Para cada 'hazard-category', ajustar un GridSearchCV específico para 'product-category'
for hazard in hazard_categories:
    hazard_idx = le_hazard.transform([hazard])[0]
    mask_train = y_hazard_train == hazard_idx

    if np.sum(mask_train) > 1:  # Verificar si hay suficientes muestras para entrenar
        X_train_hazard = X_train[mask_train]
        y_product_train_hazard = y_product_train[mask_train]

        # Crear el pipeline para 'product-category'
        pipeline_product = Pipeline([
            ('vectorizer', TfidfVectorizer()),
            ('classifier', LinearSVC())
        ])

        # Definir el espacio de búsqueda de hiperparámetros
        param_grid_product = {
            'vectorizer__max_df': [0.75, 0.85, 1.0],
            'vectorizer__min_df': [1, 2, 5],
            'vectorizer__ngram_range': [(1,1), (1,2)],
            'classifier__C': [0.01, 0.1, 1, 10],
        }

        # Crear GridSearchCV para 'product-category'
        grid_search_product = GridSearchCV(pipeline_product, param_grid_product, cv=5, scoring='f1_weighted', verbose=1, n_jobs=-1)

        # Ajustar el modelo de 'product-category'
        grid_search_product.fit(X_train_hazard, y_product_train_hazard)

        # Almacenar el mejor modelo de 'product-category'
        product_models[hazard] = grid_search_product.best_estimator_

# Paso 6: Predecir 'product-category' respetando la jerarquía
y_product_pred = []

for i in range(len(X_test)):
    hazard_idx = y_hazard_pred[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]

    if hazard_label in product_models:
        model = product_models[hazard_label]
        X_sample = [X_test.iloc[i]]
        product_pred_encoded = model.predict(X_sample)[0]
        le_product = product_encoders[hazard_label]
        product_pred_label = le_product.inverse_transform([product_pred_encoded])[0]
        y_product_pred.append((hazard_label, product_pred_label))
    else:
        y_product_pred.append((hazard_label, None))

# Evaluar el modelo para 'product-category'
y_product_true_labels = []
y_product_pred_labels = []

for i in range(len(y_product_test)):
    hazard_idx = y_hazard_test.iloc[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]
    product_idx = y_product_test.iloc[i]
    le_product = product_encoders[hazard_label]
    product_label = le_product.inverse_transform([product_idx])[0]

    y_product_true_labels.append((hazard_label, product_label))

    pred_hazard_label, pred_product_label = y_product_pred[i]
    y_product_pred_labels.append((pred_hazard_label, pred_product_label))

# Evaluar solo las instancias donde tenemos predicciones válidas
true_labels = []
pred_labels = []

for true, pred in zip(y_product_true_labels, y_product_pred_labels):
    if pred[1] is not None:
        true_labels.append(true[1])
        pred_labels.append(pred[1])

print("\nEvaluación del modelo para 'product-category' tras la optimización:")
print(classification_report(true_labels, pred_labels))
print("Matriz de confusión:")
print(confusion_matrix(true_labels, pred_labels))
print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("F1 Score:", f1_score(true_labels, pred_labels, average='weighted'))
print("Precision:", precision_score(true_labels, pred_labels, average='weighted'))
print("Recall:", recall_score(true_labels, pred_labels, average='weighted'))

In [ ]:
# Mostrar los mejores hiperparámetros para 'hazard-category'
print("Mejores hiperparámetros para 'hazard-category':")
print(grid_search_hazard.best_params_)

# Mostrar los mejores hiperparámetros para cada 'product-category'
print("\nMejores hiperparámetros para cada 'product-category':")
for hazard, model in product_models.items():
    print(f"Hazard-category: {hazard}")
    print(model.named_steps['vectorizer'], model.named_steps['classifier'])  # Mostrar el vectorizador y el clasificador
    print(model.get_params())  # Mostrar todos los parámetros del modelo
    print()  # Espacio entre modelos

# Modelo Validacion

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer  # Asegúrate de importar TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder


# Paso 1: Combinar las columnas 'title_clean' y 'text_clean' en una sola
df['combined_text'] = df['text_clean']

# Paso 2: Codificar las etiquetas
# Codificar 'hazard-category'
le_hazard = LabelEncoder()
df['hazard_encoded'] = le_hazard.fit_transform(df['hazard-category'])

# Codificar 'product-category' dentro de cada 'hazard-category'
# Crear un diccionario para almacenar los codificadores de 'product-category' por 'hazard-category'
product_encoders = {}
df['product_encoded'] = -1  # Inicializar con -1

# Obtener las categorías únicas de 'hazard-category'
hazard_categories = df['hazard-category'].unique()

for hazard in hazard_categories:
    # Filtrar el DataFrame por 'hazard-category'
    mask = df['hazard-category'] == hazard
    # Crear un LabelEncoder para el 'hazard-category' actual
    le_product = LabelEncoder()
    df.loc[mask, 'product_encoded'] = le_product.fit_transform(df.loc[mask, 'product-category'])
    # Almacenar el codificador
    product_encoders[hazard] = le_product

# Paso 3: Dividir los datos en conjuntos de entrenamiento y prueba
X = df['combined_text']
y_hazard = df['hazard_encoded']
y_product = df['product_encoded']

X_train, X_test, y_hazard_train, y_hazard_test, y_product_train, y_product_test = train_test_split(
    X, y_hazard, y_product, test_size=0.2, random_state=42, stratify=y_hazard
)

# Paso 4: Crear un Pipeline para el modelo de SVM con los hiperparámetros ajustados

# Hiperparámetros obtenidos
best_params = {
    'classifier__C': 10,
    'vectorizer__max_df': 0.65,
    'vectorizer__min_df': 2,
    # 'vectorizer__ngram_range': (1, 2)
}

# Crear el Pipeline para 'hazard-category' con TfidfVectorizer y LinearSVC usando los hiperparámetros
pipeline_hazard = Pipeline([
    ('vectorizer', TfidfVectorizer(
        max_df=best_params['vectorizer__max_df'],
        min_df=best_params['vectorizer__min_df'],
        # ngram_range=best_params['vectorizer__ngram_range']
    )),
    ('classifier', LinearSVC(C=best_params['classifier__C']))
])

# Entrenar el modelo para 'hazard-category'
pipeline_hazard.fit(X_train, y_hazard_train)

# Predecir en el conjunto de prueba
y_hazard_pred = pipeline_hazard.predict(X_test)

# Función para buscar hazard-categories válidos por año y país, retrocediendo si no hay datos
def find_valid_hazard_categories(year, country, product_hazard_per_location):
    while year >= product_hazard_per_location['year'].min():  # Limitar búsqueda al mínimo año disponible
        # Filtrar por el año y país actual
        valid_hazards = product_hazard_per_location[
            (product_hazard_per_location['year'] == year) &
            (product_hazard_per_location['country'] == country)
        ]

        if not valid_hazards.empty:
            return valid_hazards['hazard-category'].unique()  # Devolver los hazard-category válidos si hay datos

        year -= 1  # Retroceder al año anterior si no se encontraron datos

    return None  # Si no se encuentra nada, devolver None

# Función para encontrar el producto con más peso para un hazard-category dado en un año
def find_most_frequent_product_for_hazard(hazard_label, year, country, df):
    # Filtrar el DataFrame por el hazard-category y el año específico
    mask = (df['hazard-category'] == hazard_label) & (df['year'] == year) & (df['country'] == country)
    filtered_df = df[mask]

    if not filtered_df.empty:
        # Encontrar el producto más frecuente en ese hazard-category
        most_frequent_product = filtered_df['product-category'].mode()[0]  # Devuelve el producto más frecuente
        return most_frequent_product
    else:
        # Si no hay productos en ese año, retrocedemos para buscar el producto más frecuente históricamente
        mask_all_years = df['hazard-category'] == hazard_label
        filtered_df_all_years = df[mask_all_years]
        if not filtered_df_all_years.empty:
            most_frequent_product = filtered_df_all_years['product-category'].mode()[0]
            return most_frequent_product
        else:
            return None  # Si no hay datos históricos, devolver None

# Paso 5: Evaluar el modelo para 'hazard-category'
print("Evaluación del modelo para 'hazard-category':")
print(classification_report(y_hazard_test, y_hazard_pred, target_names=le_hazard.classes_))
print("Matriz de confusión:")
print(confusion_matrix(y_hazard_test, y_hazard_pred))
print("Accuracy:", accuracy_score(y_hazard_test, y_hazard_pred))
print("F1 Score:", f1_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Precision:", precision_score(y_hazard_test, y_hazard_pred, average='weighted'))
print("Recall:", recall_score(y_hazard_test, y_hazard_pred, average='weighted'))

# Paso 6: Para 'product-category', necesitamos tener en cuenta la jerarquía
# Crear un diccionario para almacenar los modelos por 'hazard-category'
product_models = {}

# Entrenar un modelo para cada 'hazard-category' con los mismos hiperparámetros
for hazard in hazard_categories:
    # Filtrar los datos correspondientes
    hazard_idx = le_hazard.transform([hazard])[0]
    mask_train = y_hazard_train == hazard_idx
    # Verificar si hay suficientes muestras para entrenar
    if np.sum(mask_train) > 1:
        X_train_hazard = X_train[mask_train]
        y_product_train_hazard = y_product_train[mask_train]

        # Crear y entrenar el modelo con TfidfVectorizer y LinearSVC usando los hiperparámetros
        pipeline_product = Pipeline([
            ('vectorizer', TfidfVectorizer(
                max_df=best_params['vectorizer__max_df'],
                min_df=best_params['vectorizer__min_df'],
                # ngram_range=best_params['vectorizer__ngram_range']
            )),
            ('classifier', LinearSVC(C=best_params['classifier__C']))
        ])
        pipeline_product.fit(X_train_hazard, y_product_train_hazard)
        # Almacenar el modelo
        product_models[hazard] = pipeline_product

# Paso 7: Predecir 'product-category' teniendo en cuenta 'hazard-category' y validación por año/país
y_product_pred = []

# product_hazard_per_location tiene todas las combinaciones de year, country y hazard
product_hazard_per_location = df.groupby(['year', 'country', 'product-category'])['hazard-category'].nunique().reset_index()

# Necesitamos los valores de 'year' y 'country' para X_test
X_test_indices = X_test.index
year_test = df.loc[X_test_indices, 'year'].reset_index(drop=True)
country_test = df.loc[X_test_indices, 'country'].reset_index(drop=True)

for i in range(len(X_test)):
    hazard_idx = y_hazard_pred[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]

    # Si tenemos un modelo para este 'hazard-category'
    if hazard_label in product_models:
        model = product_models[hazard_label]
        # Transformar el texto en una lista para el modelo
        X_sample = [X_test.iloc[i]]
        # Predecir 'product-category'
        product_pred_encoded = model.predict(X_sample)[0]
        # Decodificar la predicción
        le_product = product_encoders[hazard_label]
        product_pred_label = le_product.inverse_transform([product_pred_encoded])[0]
        # Añadir a la lista de predicciones
        y_product_pred.append((hazard_label, product_pred_label))
    else:
        # Si no hay modelo, validar año y país, y asignar un hazard válido
        year_pred = year_test.iloc[i]  # Año actual
        country_pred = country_test.iloc[i]  # País actual

        valid_hazard_categories = find_valid_hazard_categories(year_pred, country_pred, product_hazard_per_location)

        if valid_hazard_categories is not None:
            # Devolver el producto con más peso para este hazard-category en ese año/país
            most_frequent_product = find_most_frequent_product_for_hazard(hazard_label, year_pred, country_pred, df)
            y_product_pred.append((hazard_label, most_frequent_product))
        else:
            # Si no se encuentran categorías válidas, asignar el producto más frecuente para ese hazard
            most_frequent_product = find_most_frequent_product_for_hazard(hazard_label, year_pred, country_pred, df)
            y_product_pred.append((hazard_label, most_frequent_product))

# Paso 8: Evaluar el modelo para 'product-category'
y_product_true_labels = []
y_product_pred_labels = []

for i in range(len(y_product_test)):
    hazard_idx = y_hazard_test.iloc[i]
    hazard_label = le_hazard.inverse_transform([hazard_idx])[0]
    product_idx = y_product_test.iloc[i]
    le_product = product_encoders[hazard_label]
    product_label = le_product.inverse_transform([product_idx])[0]

    y_product_true_labels.append((hazard_label, product_label))

    # Obtener la predicción correspondiente
    pred_hazard_label, pred_product_label = y_product_pred[i]
    y_product_pred_labels.append((pred_hazard_label, pred_product_label))

# Evaluar solo las instancias donde tenemos predicciones válidas
true_labels = []
pred_labels = []

for true, pred in zip(y_product_true_labels, y_product_pred_labels):
    # Si la predicción no es None
    if pred[1] is not None:
        true_labels.append(true[1])
        pred_labels.append(pred[1])

# Reporte de clasificación para 'product-category'
print("\nEvaluación del modelo para 'product-category':")
print(classification_report(true_labels, pred_labels))
print("Matriz de confusión:")
print(confusion_matrix(true_labels, pred_labels))
print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("F1 Score:", f1_score(true_labels, pred_labels, average='weighted'))
print("Precision:", precision_score(true_labels, pred_labels, average='weighted'))
print("Recall:", recall_score(true_labels, pred_labels, average='weighted'))